In [1]:
import os
import numpy as np
import pandas as pd
import nasdaqdatalink
from xgboost import XGBRegressor

pd.set_option("mode.copy_on_write", True)

SEED = 7
np.random.seed(SEED)

START = "2005-01-01"
END = "2025-12-31"

MIN_PRICE = 5.0
MIN_DVOL_20 = 5_000_000

N_LONG = 50
N_SHORT = 50

COST_BPS_ONE_WAY = 5.0
FUNDAMENTALS_LAG_TRADING_DAYS = 1

OUTPUT_XLSX = "monthly_holdings.xlsx"
CACHE_DIR = "cache_sharadar"
os.makedirs(CACHE_DIR, exist_ok=True)
nasdaqdatalink.ApiConfig.api_key = "vo3osJWr68eTVPawaV_B"


In [2]:
#functions

from pathlib import Path

def _chunk(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def _month_end_trading_days(trading_days_index):
    s = pd.Series(trading_days_index, index=trading_days_index)
    return s.groupby(s.index.to_period("M")).max().sort_values().tolist()

def _next_trading_day(trading_days_index, d):
    d = pd.Timestamp(d)
    pos = trading_days_index.searchsorted(d)
    if pos >= len(trading_days_index) - 1:
        return None
    if trading_days_index[pos] == d:
        return trading_days_index[pos + 1]
    return trading_days_index[pos]

def _shift_trading_days(trading_days_index, d, k):
    d = pd.Timestamp(d)
    pos = trading_days_index.searchsorted(d)
    if pos >= len(trading_days_index):
        return None
    if trading_days_index[pos] != d:
        pos -= 1
    pos2 = pos + int(k)
    if pos2 < 0 or pos2 >= len(trading_days_index):
        return None
    return trading_days_index[pos2]

def _save_parquet(df, path):
    df.to_parquet(path, index=False)

def _load_parquet(path):
    return pd.read_parquet(path)

def fetch_tickers_universe(table="SEP"):
    path = Path(CACHE_DIR) / f"tickers_{table}.parquet"
    if path.exists():
        return _load_parquet(path)

    df = nasdaqdatalink.get_table("SHARADAR/TICKERS", table=table, paginate=True)
    if df is None or df.empty:
        raise ValueError("SHARADAR/TICKERS returned no rows.")
    if "ticker" not in df.columns:
        raise ValueError("SHARADAR/TICKERS did not return 'ticker'.")

    if "category" in df.columns:
        df = df[df["category"].isin(["Domestic Common Stock", "Domestic Common Stock Primary", "Domestic Common Stock Secondary"])]

    df = df.dropna(subset=["ticker"]).reset_index(drop=True)
    _save_parquet(df, path)
    return df

def build_trade_universe(univ, start, end):
    u = univ.copy()
    start_dt = pd.Timestamp(start)
    end_dt = pd.Timestamp(end)

    if "firstpricedate" in u.columns:
        u["firstpricedate"] = pd.to_datetime(u["firstpricedate"], errors="coerce")
    if "lastpricedate" in u.columns:
        u["lastpricedate"] = pd.to_datetime(u["lastpricedate"], errors="coerce")

    if "firstpricedate" in u.columns and "lastpricedate" in u.columns:
        u = u[(u["firstpricedate"] <= end_dt) & (u["lastpricedate"] >= start_dt)]

    return u["ticker"].dropna().unique().tolist()

def fetch_sep_prices_yearly(tickers, start, end, columns=None, chunk_size=150, cache_name="sep_full"):
    start_dt = pd.Timestamp(start)
    end_dt = pd.Timestamp(end)

    if columns is None:
        columns = ["ticker", "date", "closeadj", "close", "volume"]

    path = Path(CACHE_DIR) / f"{cache_name}_{start_dt.date()}_{end_dt.date()}_c{chunk_size}.parquet"
    if path.exists():
        return _load_parquet(path)

    years = range(start_dt.year, end_dt.year + 1)
    out = []

    for y in years:
        w0 = max(pd.Timestamp(f"{y}-01-01"), start_dt)
        w1 = min(pd.Timestamp(f"{y}-12-31"), end_dt)
        if w0 > w1:
            continue

        w0s = str(w0.date())
        w1s = str(w1.date())

        for tk_chunk in _chunk(list(tickers), chunk_size):
            df = nasdaqdatalink.get_table(
                "SHARADAR/SEP",
                ticker=tk_chunk,
                date={"gte": w0s, "lte": w1s},
                qopts={"columns": columns},
                paginate=True
            )
            if df is not None and not df.empty:
                out.append(df)

    px = pd.concat(out, ignore_index=True) if out else pd.DataFrame()
    if px.empty:
        raise ValueError("SEP pull returned no rows. Check tickers/date range.")

    px["date"] = pd.to_datetime(px["date"])
    _save_parquet(px, path)
    return px

def fetch_sfp_prices_yearly(tickers, start, end, columns=None, chunk_size=200, cache_name="sfp_full"):
    start_dt = pd.Timestamp(start)
    end_dt = pd.Timestamp(end)

    if columns is None:
        columns = ["ticker", "date", "closeadj", "close", "volume"]

    path = Path(CACHE_DIR) / f"{cache_name}_{start_dt.date()}_{end_dt.date()}_c{chunk_size}.parquet"
    if path.exists():
        return _load_parquet(path)

    years = range(start_dt.year, end_dt.year + 1)
    out = []

    for y in years:
        w0 = max(pd.Timestamp(f"{y}-01-01"), start_dt)
        w1 = min(pd.Timestamp(f"{y}-12-31"), end_dt)
        if w0 > w1:
            continue

        w0s = str(w0.date())
        w1s = str(w1.date())

        for tk_chunk in _chunk(list(tickers), chunk_size):
            df = nasdaqdatalink.get_table(
                "SHARADAR/SFP",
                ticker=tk_chunk,
                date={"gte": w0s, "lte": w1s},
                qopts={"columns": columns},
                paginate=True
            )
            if df is not None and not df.empty:
                out.append(df)

    px = pd.concat(out, ignore_index=True) if out else pd.DataFrame()
    if px.empty:
        raise ValueError("SFP pull returned no rows. Check entitlement/tickers/date range.")

    px["date"] = pd.to_datetime(px["date"])
    _save_parquet(px, path)
    return px

def fetch_sf1_fundamentals_all_numeric(tickers, start, end, dimension="ART", chunk_size=200, cache_name="sf1_allnum"):
    start_s = str(pd.Timestamp(start).date())
    end_s = str(pd.Timestamp(end).date())

    path = Path(CACHE_DIR) / f"{cache_name}_{dimension}_{start_s}_{end_s}_c{chunk_size}.parquet"
    if path.exists():
        return _load_parquet(path)

    out = []
    for tk_chunk in _chunk(list(tickers), chunk_size):
        df = nasdaqdatalink.get_table(
            "SHARADAR/SF1",
            ticker=tk_chunk,
            dimension=dimension,
            datekey={"gte": start_s, "lte": end_s},
            paginate=True
        )
        if df is not None and not df.empty:
            out.append(df)

    f = pd.concat(out, ignore_index=True) if out else pd.DataFrame()
    if f.empty:
        return f

    f["datekey"] = pd.to_datetime(f["datekey"], errors="coerce")
    if "calendardate" in f.columns:
        f["calendardate"] = pd.to_datetime(f["calendardate"], errors="coerce")

    keep_id = {"ticker", "dimension", "datekey", "calendardate"}
    num_cols = [c for c in f.columns if c in keep_id or pd.api.types.is_numeric_dtype(f[c])]
    f = f[num_cols]

    _save_parquet(f, path)
    return f

def pick_benchmark(as_of_date):
    d = str(pd.Timestamp(as_of_date).date())

    for tk in ["SPX", "^GSPC"]:
        try:
            df = nasdaqdatalink.get_table(
                "SHARADAR/SEP",
                ticker=tk,
                date={"eq": d},
                qopts={"columns": ["ticker", "date", "close", "closeadj"]},
                paginate=False
            )
            if df is not None and not df.empty:
                return ("SEP", tk)
        except Exception:
            pass

    for tk in ["SPY", "IVV", "VOO"]:
        try:
            df = nasdaqdatalink.get_table(
                "SHARADAR/SFP",
                ticker=tk,
                date={"eq": d},
                qopts={"columns": ["ticker", "date", "close", "closeadj"]},
                paginate=False
            )
            if df is not None and not df.empty:
                return ("SFP", tk)
        except Exception:
            pass

    return ("SFP", "SPY")

univ = fetch_tickers_universe(table="SEP")
tickers = build_trade_universe(univ, START, END)

BENCH_SOURCE, BENCH_TICKER = pick_benchmark(END)
BENCH_SOURCE, BENCH_TICKER


('SFP', 'SPY')

In [ ]:
# --- backtest config + modular functions (long/short) ---
import numpy as np
import pandas as pd
import xgboost as xgb

CONFIG = dict(
    SEED=7,
    N_LONG=50,
    N_SHORT=50,
    TRAIN_Q=12,
    VAL_Q=4,
    TEST_Q=4,
    STEP_Q=4,
    MIN_PRICE=5.0,
    MIN_DVOL_20=1_000_000,
    COST_BPS_ONE_WAY=5.0,
    RET_CLIP_LOWER=-0.8,
    RET_CLIP_UPPER=1.5,
    USE_RET_CLIP=False,
)


def make_folds(q_end_dates, train_q, val_q, test_q, step_q):
    q_end = pd.DatetimeIndex(sorted(pd.to_datetime(q_end_dates)))
    total = train_q + val_q + test_q
    folds = []
    k = 0
    for i in range(0, len(q_end) - total + 1, step_q):
        train_dates = q_end[i : i + train_q]
        val_dates = q_end[i + train_q : i + train_q + val_q]
        test_dates = q_end[i + train_q + val_q : i + total]
        folds.append(
            {
                "fold": k,
                "train_dates": list(train_dates),
                "val_dates": list(val_dates),
                "test_dates": list(test_dates),
            }
        )
        k += 1
    return folds


def build_fold_datasets(Xq, y_ret3m, folds):
    if not isinstance(Xq.index, pd.MultiIndex):
        raise ValueError("Xq must use a MultiIndex (date, ticker).")
    if not isinstance(y_ret3m.index, pd.MultiIndex):
        raise ValueError("y_ret3m must use a MultiIndex (date, ticker).")

    Xq = Xq.sort_index()
    y_ret3m = y_ret3m.sort_index()

    datasets = []
    for fold in folds:
        train_dates = fold["train_dates"]
        val_dates = fold["val_dates"]
        test_dates = fold["test_dates"]

        X_train = Xq.loc[(train_dates, slice(None))].sort_index()
        y_train = y_ret3m.loc[(train_dates, slice(None))].reindex(X_train.index)

        X_val = Xq.loc[(val_dates, slice(None))].sort_index()
        y_val = y_ret3m.loc[(val_dates, slice(None))].reindex(X_val.index)

        X_test = Xq.loc[(test_dates, slice(None))].sort_index()
        y_test = y_ret3m.loc[(test_dates, slice(None))].reindex(X_test.index)

        datasets.append(
            {
                "fold": fold["fold"],
                "X_train": X_train,
                "y_train": y_train,
                "X_val": X_val,
                "y_val": y_val,
                "X_test": X_test,
                "y_test": y_test,
            }
        )
    return datasets


def _zscore_by_date(y):
    def _z(s):
        mu = s.mean()
        sigma = s.std(ddof=0)
        if sigma == 0 or np.isnan(sigma):
            return pd.Series(0.0, index=s.index)
        return (s - mu) / sigma

    return y.groupby(level=0).transform(_z)


def _rank_corr(x, y):
    if len(x) < 2:
        return np.nan
    x_rank = x.rank(method="average")
    y_rank = y.rank(method="average")
    return x_rank.corr(y_rank)


def train_xgb_fold(
    X_train,
    y_train,
    X_val,
    y_val,
    feat_cols,
    params,
    early_stop=50,
    num_rounds=500,
):
    X_train = X_train.sort_index()
    X_val = X_val.sort_index()

    y_train_z = _zscore_by_date(y_train)

    dtrain = xgb.DMatrix(X_train[feat_cols], label=y_train_z)
    dval = xgb.DMatrix(X_val[feat_cols], label=_zscore_by_date(y_val))

    p = dict(params)
    p.setdefault("nthread", 1)
    p.setdefault("tree_method", "hist")
    p.setdefault("seed", CONFIG["SEED"])

    evals_result = {}
    booster = xgb.train(
        p,
        dtrain,
        num_boost_round=num_rounds,
        evals=[(dval, "val")],
        early_stopping_rounds=early_stop,
        evals_result=evals_result,
        verbose_eval=False,
    )

    best_iteration = booster.best_iteration
    trained_rounds = booster.best_iteration + 1
    val_rmse = evals_result["val"]["rmse"]
    val_rmse_best = float(val_rmse[best_iteration]) if val_rmse else np.nan
    rmse_improve = float(val_rmse[0]) - val_rmse_best if val_rmse else np.nan

    preds = booster.predict(dval, iteration_range=(0, trained_rounds))
    pred_s = pd.Series(preds, index=X_val.index)
    y_val_s = y_val.reindex(X_val.index)

    ic_by_date = []
    for d, grp in pred_s.groupby(level=0):
        y_grp = y_val_s.loc[(d, slice(None))]
        x = grp.droplevel(0)
        y = y_grp.droplevel(0)
        ic_by_date.append(_rank_corr(x, y))
    val_ic = float(np.nanmean(ic_by_date)) if ic_by_date else np.nan

    return {
        "booster": booster,
        "best_iteration": int(best_iteration),
        "trained_rounds": int(trained_rounds),
        "val_rmse_best": val_rmse_best,
        "val_ic": val_ic,
        "rmse_improve": rmse_improve,
    }


def filter_universe(Xsig, pred, ysig, min_price, min_dvol):
    if not isinstance(Xsig, pd.DataFrame):
        raise ValueError("Xsig must be a DataFrame.")
    required = {"px", "dvol_20"}
    missing = required - set(Xsig.columns)
    if missing:
        raise ValueError(f"Xsig is missing columns: {sorted(missing)}")

    mask = (Xsig["px"] >= min_price) & (Xsig["dvol_20"] >= min_dvol)
    tickers = Xsig.index[mask]

    pred_u = pred.reindex(tickers)
    y_u = ysig.reindex(tickers)

    universe_meta = {"n_universe": int(len(tickers))}
    return pred_u, y_u, universe_meta


def select_portfolio(pred_u, n_long, n_short):
    df = pred_u.dropna().reset_index()
    df.columns = ["ticker", "pred"]
    df = df.sort_values(["pred", "ticker"], ascending=[True, True], kind="mergesort")

    shorts = df.head(n_short)["ticker"].tolist() if n_short > 0 else []
    longs = df.tail(n_long)["ticker"].tolist() if n_long > 0 else []
    return longs, shorts


def compute_weights(longs, shorts):
    weights = {}
    if longs:
        w_long = 1.0 / len(longs)
        weights.update({t: w_long for t in longs})
    if shorts:
        w_short = -1.0 / len(shorts)
        weights.update({t: w_short for t in shorts})
    return pd.Series(weights, name="weight")


def compute_turnover(prev_w, w):
    if prev_w is None or prev_w.empty:
        return 0.5 * float(w.abs().sum())
    w_all = prev_w.reindex(prev_w.index.union(w.index)).fillna(0.0)
    w_new = w.reindex(w_all.index).fillna(0.0)
    return 0.5 * float((w_new - w_all).abs().sum())


def run_backtest(fold_datasets, models, feat_cols, config):
    perf_rows = []
    weights_rows = []
    holdings_rows = []
    universe_rows = []

    for fd in fold_datasets:
        k = fd["fold"]
        model = models[k]["booster"]
        X_test = fd["X_test"].sort_index()
        y_test = fd["y_test"].sort_index()

        prev_w = None
        for d in X_test.index.get_level_values(0).unique():
            Xsig = X_test.xs(d, level=0).sort_index()
            ysig = y_test.xs(d, level=0).sort_index()

            pred = pd.Series(
                model.predict(xgb.DMatrix(Xsig[feat_cols])),
                index=Xsig.index,
                name="pred",
            )

            pred_u, y_u, meta = filter_universe(
                Xsig,
                pred,
                ysig,
                config["MIN_PRICE"],
                config["MIN_DVOL_20"],
            )

            if config.get("USE_RET_CLIP", False):
                y_u = y_u.clip(config["RET_CLIP_LOWER"], config["RET_CLIP_UPPER"])

            longs, shorts = select_portfolio(pred_u, config["N_LONG"], config["N_SHORT"])
            w = compute_weights(longs, shorts)

            turnover = compute_turnover(prev_w, w)
            cost = turnover * (config["COST_BPS_ONE_WAY"] / 10_000)
            gross_ret = float((w * y_u.reindex(w.index)).sum()) if not w.empty else 0.0
            net_ret = gross_ret - cost

            perf_rows.append(
                {
                    "fold": k,
                    "signal_date": pd.Timestamp(d),
                    "gross_ret": gross_ret,
                    "net_ret": net_ret,
                    "turnover": turnover,
                    "n_universe": meta["n_universe"],
                }
            )

            for t, wt in w.items():
                holdings_rows.append(
                    {
                        "fold": k,
                        "signal_date": pd.Timestamp(d),
                        "ticker": t,
                        "side": "long" if wt > 0 else "short",
                        "weight": wt,
                        "pred": pred_u.get(t, np.nan),
                        "asset_return": y_u.get(t, np.nan),
                    }
                )

            for t, pr in pred_u.items():
                universe_rows.append(
                    {
                        "fold": k,
                        "signal_date": pd.Timestamp(d),
                        "ticker": t,
                        "pred": pr,
                        "asset_return": y_u.get(t, np.nan),
                    }
                )

            for t, wt in w.items():
                weights_rows.append(
                    {
                        "fold": k,
                        "signal_date": pd.Timestamp(d),
                        "ticker": t,
                        "weight": wt,
                    }
                )

            prev_w = w

    perf = pd.DataFrame(perf_rows)
    weights = pd.DataFrame(weights_rows)
    holdings = pd.DataFrame(holdings_rows)
    universe_panel = pd.DataFrame(universe_rows)

    return {
        "perf": perf,
        "weights": weights,
        "holdings": holdings,
        "universe_panel": universe_panel,
    }


def dedupe_live(perf, weights, holdings):
    perf = perf.sort_values(["signal_date", "fold"])
    latest = perf.groupby("signal_date")["fold"].idxmax()
    perf_live = perf.loc[latest].reset_index(drop=True)

    keep = perf_live[["signal_date", "fold"]]
    weights_live = weights.merge(keep, on=["signal_date", "fold"], how="inner")
    holdings_live = holdings.merge(keep, on=["signal_date", "fold"], how="inner")

    return {
        "perf_live": perf_live,
        "weights_live": weights_live,
        "holdings_live": holdings_live,
    }


def ic_timeseries(panel):
    rows = []
    for keys, grp in panel.groupby(["signal_date", "fold"]):
        d, f = keys
        grp = grp.dropna(subset=["pred", "asset_return"])
        if len(grp) < 2:
            continue
        ic = _rank_corr(grp["pred"], grp["asset_return"])
        rows.append({"signal_date": d, "fold": f, "ic": ic})

    ic_df = pd.DataFrame(rows)
    if not ic_df.empty:
        mean_ic = ic_df["ic"].mean()
        t_stat = mean_ic / ic_df["ic"].std(ddof=1) * np.sqrt(len(ic_df)) if len(ic_df) > 1 else np.nan
        ic_df.attrs.update({"mean_ic": mean_ic, "t_stat": t_stat})
    return ic_df


def decile_curve(universe_panel):
    rows = []
    for keys, grp in universe_panel.groupby(["signal_date", "fold"]):
        d, f = keys
        grp = grp.dropna(subset=["pred", "asset_return"])
        if len(grp) < 10:
            continue
        grp = grp.copy()
        grp["decile"] = pd.qcut(grp["pred"], 10, labels=False, duplicates="drop")
        deci = grp.groupby("decile")["asset_return"].mean()
        for dec, val in deci.items():
            rows.append({"signal_date": d, "fold": f, "decile": int(dec), "ret": float(val)})

    decile_df = pd.DataFrame(rows)
    summary = None
    if not decile_df.empty:
        summary = decile_df.groupby("decile")["ret"].mean().rename("mean_ret")
        top_bottom = summary.loc[summary.index.max()] - summary.loc[summary.index.min()]
        summary = summary.to_frame()
        summary.loc["top_bottom", "mean_ret"] = top_bottom
    return decile_df, summary


def ls_spread_timeseries(universe_panel):
    rows = []
    for keys, grp in universe_panel.groupby(["signal_date", "fold"]):
        d, f = keys
        grp = grp.dropna(subset=["pred", "asset_return"])
        if len(grp) < 10:
            continue
        grp = grp.copy()
        grp["decile"] = pd.qcut(grp["pred"], 10, labels=False, duplicates="drop")
        top = grp[grp["decile"] == grp["decile"].max()]["asset_return"].mean()
        bot = grp[grp["decile"] == grp["decile"].min()]["asset_return"].mean()
        rows.append({"signal_date": d, "fold": f, "ls_spread": top - bot})

    ls_df = pd.DataFrame(rows)
    if not ls_df.empty:
        mean_ls = ls_df["ls_spread"].mean()
        t_stat = mean_ls / ls_df["ls_spread"].std(ddof=1) * np.sqrt(len(ls_df)) if len(ls_df) > 1 else np.nan
        ls_df.attrs.update({"mean_ls": mean_ls, "t_stat": t_stat})
    return ls_df


def turnover_report(perf_live):
    df = perf_live.dropna(subset=["turnover", "net_ret"]).copy()
    if df.empty:
        return pd.DataFrame()
    df["turnover_q"] = pd.qcut(df["turnover"], 4, labels=False, duplicates="drop")
    report = df.groupby("turnover_q").agg(
        mean_turnover=("turnover", "mean"),
        mean_net_ret=("net_ret", "mean"),
        count=("net_ret", "count"),
    )
    return report


def cost_sensitivity(perf_inputs, cost_grid=(0, 5, 10, 20, 50)):
    if "gross_ret" not in perf_inputs or "turnover" not in perf_inputs:
        raise ValueError("perf_inputs must include gross_ret and turnover columns.")
    rows = []
    for cost in cost_grid:
        net = perf_inputs["gross_ret"] - perf_inputs["turnover"] * (cost / 10_000)
        rows.append({"cost_bps": cost, "mean_net_ret": net.mean()})
    return pd.DataFrame(rows)


def export_quarter_holdings_excel(holdings_live, date, path, n_long, n_short):
    d = pd.Timestamp(date)
    df = holdings_live[holdings_live["signal_date"] == d]
    if df.empty:
        raise ValueError("No holdings found for the requested date.")
    longs = df[df["side"] == "long"].nlargest(n_long, "weight")
    shorts = df[df["side"] == "short"].nsmallest(n_short, "weight")

    with pd.ExcelWriter(path) as writer:
        longs[["ticker", "weight", "asset_return"]].to_excel(writer, sheet_name="LONG", index=False)
        shorts[["ticker", "weight", "asset_return"]].to_excel(writer, sheet_name="SHORT", index=False)


In [8]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT.name != "Bison" and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent

SEP_CACHE_DIR = (PROJECT_ROOT / "cache_sharadar" / "sep_by_year").resolve()

def _load_year(y):
    p = SEP_CACHE_DIR / f"sep_{y}.parquet"
    if not p.exists():
        return None
    df = pd.read_parquet(p)
    if df is None or df.empty:
        return None
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["ticker", "date"])
    df["ticker"] = df["ticker"].astype(str)
    return df

def load_sep_from_year_cache(start, end):
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    years = range(start.year, end.year + 1)

    dfs = []
    missing = []
    for y in years:
        df = _load_year(y)
        if df is None:
            missing.append(y)
            continue
        df = df[(df["date"] >= start) & (df["date"] <= end)]
        if not df.empty:
            dfs.append(df)

    if not dfs:
        raise ValueError(f"No SEP data loaded. Missing years: {missing}. Dir: {SEP_CACHE_DIR}")

    px = pd.concat(dfs, ignore_index=True)
    px = px.drop_duplicates(subset=["ticker", "date"], keep="last")
    px = px.sort_values(["date", "ticker"]).reset_index(drop=True)
    return px

px_stocks = load_sep_from_year_cache(START, END)
print("px_stocks:", px_stocks.shape, px_stocks["date"].min(), px_stocks["date"].max(), px_stocks["ticker"].nunique())


px_stocks: (21474668, 5) 2005-01-03 00:00:00 2025-12-19 00:00:00 9306


In [9]:
# build trading calendar

import pandas as pd
import numpy as np

def _month_end_trading_days(trading_days_index):
    s = pd.Series(trading_days_index, index=trading_days_index)
    return s.groupby(s.index.to_period("M")).max().sort_values().tolist()

def _next_trading_day(trading_days_index, d):
    d = pd.Timestamp(d)
    pos = trading_days_index.searchsorted(d)
    if pos >= len(trading_days_index) - 1:
        return None
    if trading_days_index[pos] == d:
        return trading_days_index[pos + 1]
    return trading_days_index[pos]

def _pick_price_col(df):
    if "closeadj" in df.columns:
        return "closeadj"
    if "close" in df.columns:
        return "close"
    raise ValueError("No close/closeadj column found in SEP data.")

# px_stocks is your loaded SEP dataframe
px_stocks["date"] = pd.to_datetime(px_stocks["date"])
price_col = _pick_price_col(px_stocks)

stock_close = px_stocks.pivot(index="date", columns="ticker", values=price_col).sort_index()
stock_vol = None
if "volume" in px_stocks.columns:
    stock_vol = px_stocks.pivot(index="date", columns="ticker", values="volume").sort_index()

all_days = stock_close.index.sort_values()

month_ends = _month_end_trading_days(all_days)
month_ends = [d for d in month_ends if (pd.Timestamp(START) <= pd.Timestamp(d) <= pd.Timestamp(END))]

pairs = []
for i in range(len(month_ends) - 1):
    sig = pd.Timestamp(month_ends[i])                 # signal at EOM close
    entry = _next_trading_day(all_days, sig)          # trade next trading day close
    next_sig = pd.Timestamp(month_ends[i + 1])        # next EOM close
    exit_ = _next_trading_day(all_days, next_sig)     # exit next trading day close after next EOM
    if entry is None or exit_ is None:
        continue
    pairs.append((sig, entry, exit_))

if not pairs:
    raise ValueError("No valid monthly entry/exit pairs found. Check START/END vs cached SEP dates.")

signal_dates = [p[0] for p in pairs]

print("Trading days:", len(all_days))
print("Months in backtest:", len(pairs))
print("Signal range:", signal_dates[0].date(), "->", signal_dates[-1].date())
pairs[:3], pairs[-3:]


Trading days: 5276
Months in backtest: 250
Signal range: 2005-01-31 -> 2025-10-31


([(Timestamp('2005-01-31 00:00:00'),
   Timestamp('2005-02-01 00:00:00'),
   Timestamp('2005-03-01 00:00:00')),
  (Timestamp('2005-02-28 00:00:00'),
   Timestamp('2005-03-01 00:00:00'),
   Timestamp('2005-04-01 00:00:00')),
  (Timestamp('2005-03-31 00:00:00'),
   Timestamp('2005-04-01 00:00:00'),
   Timestamp('2005-05-02 00:00:00'))],
 [(Timestamp('2025-08-29 00:00:00'),
   Timestamp('2025-09-02 00:00:00'),
   Timestamp('2025-10-01 00:00:00')),
  (Timestamp('2025-09-30 00:00:00'),
   Timestamp('2025-10-01 00:00:00'),
   Timestamp('2025-11-03 00:00:00')),
  (Timestamp('2025-10-31 00:00:00'),
   Timestamp('2025-11-03 00:00:00'),
   Timestamp('2025-12-01 00:00:00'))])

In [10]:
# ---- inputs assumed already defined ----
# stock_close (date x ticker), stock_vol (date x ticker or None)
# pairs (list of (signal_date, entry_date, exit_date))
# START, END, MIN_PRICE, MIN_DVOL_20

pairs_df = pd.DataFrame(pairs, columns=["signal_date", "trade_date", "exit_date"]).set_index("signal_date").sort_index()

# forward returns for every ticker for each month (entry close -> exit close)
trade_px = stock_close.reindex(pairs_df["trade_date"].values)
exit_px  = stock_close.reindex(pairs_df["exit_date"].values)

trade_px.index = pairs_df.index
exit_px.index  = pairs_df.index

fwd_stock = (exit_px / trade_px - 1.0).astype("float32")
fwd_stock.index.name = "signal_date"

# ---- strict eligibility at signal date (EOM close) ----
sig_px = stock_close.reindex(pairs_df.index)

eligible = (sig_px >= MIN_PRICE)

if stock_vol is not None:
    dvol_20_daily = (stock_vol * stock_close).rolling(20, min_periods=20).mean()
    dvol_20_sig = dvol_20_daily.reindex(pairs_df.index)
    eligible = eligible & (dvol_20_sig >= MIN_DVOL_20)

# extra strict: require valid trade/exit prices too (prevents weird division artifacts)
eligible = eligible & trade_px.notna() & exit_px.notna()

# diagnostics (you already liked these)
elig_counts = eligible.sum(axis=1).astype(int)
print("Eligible names per month:")
print(elig_counts.describe())

low_cut = int(elig_counts.quantile(0.05))
print("\nMonths below 5th pct threshold (", low_cut, "):")
print(elig_counts[elig_counts <= low_cut].head(30))

# ---- synthetic benchmark = average eligible forward return each month ----
fwd_bench = fwd_stock.where(eligible).mean(axis=1).astype("float32")
fwd_bench.name = "bench_fwd"

# ---- excess forward return target y(date,ticker) ----
y_excess = fwd_stock.sub(fwd_bench, axis=0).stack(future_stack=True).astype("float32")
y_excess.index.set_names(["date", "ticker"], inplace=True)
y_excess.name = "y"

print("\nShapes:")
print("pairs_df:", pairs_df.shape)
print("fwd_stock:", fwd_stock.shape)
print("fwd_bench NaNs:", int(fwd_bench.isna().sum()))
print("y_excess rows:", int(len(y_excess)))


Eligible names per month:
count     250.000000
mean     1562.176000
std       210.099367
min      1021.000000
25%      1413.500000
50%      1594.500000
75%      1699.750000
max      2108.000000
dtype: float64

Months below 5th pct threshold ( 1201 ):
signal_date
2008-11-28    1101
2008-12-31    1060
2009-01-30    1048
2009-02-27    1021
2009-03-31    1036
2009-04-30    1126
2009-05-29    1181
2009-06-30    1178
2009-07-31    1164
2009-08-31    1185
2009-11-30    1200
2009-12-31    1201
2010-07-30    1201
2010-08-31    1177
dtype: int64

Shapes:
pairs_df: (250, 2)
fwd_stock: (250, 9306)
fwd_bench NaNs: 0
y_excess rows: 2326500


In [11]:
# build price and vol features
# -----------------------------
# Build PRICE features at signal dates (and cache to parquet)
# -----------------------------

cache_path = Path(CACHE_DIR) / f"X_price_{pd.Timestamp(START).date()}_{pd.Timestamp(END).date()}.parquet"

if cache_path.exists():
    X_price = pd.read_parquet(cache_path)
    X_price.index = pd.MultiIndex.from_frame(
        X_price.index.to_frame(index=False).rename(columns={0: "date", 1: "ticker"})
    ) if not isinstance(X_price.index, pd.MultiIndex) else X_price.index
    X_price.index.set_names(["date", "ticker"], inplace=True)
    print("Loaded cached X_price:", X_price.shape)
else:
    stock_close = stock_close.sort_index()
    if stock_vol is not None:
        stock_vol = stock_vol.reindex(stock_close.index).sort_index()

    sig = pd.Index(pd.to_datetime(signal_dates)).unique().sort_values()
    sig = sig.intersection(stock_close.index)

    print("signal months:", len(sig))
    print("tickers:", stock_close.shape[1])
    print("price range:", stock_close.index.min(), "->", stock_close.index.max())

    daily_ret = stock_close.pct_change(fill_method=None)

    ret_21  = stock_close / stock_close.shift(21)  - 1.0
    ret_63  = stock_close / stock_close.shift(63)  - 1.0
    ret_126 = stock_close / stock_close.shift(126) - 1.0
    ret_252 = stock_close / stock_close.shift(252) - 1.0

    vol_63  = daily_ret.rolling(63).std()
    vol_252 = daily_ret.rolling(252).std()

    dvol_20 = None
    if stock_vol is not None:
        dvol_20 = (stock_vol * stock_close).rolling(20).mean()

    def _stack_at(mat, name):
        tmp = mat.loc[sig].stack(future_stack=True).rename(name).to_frame()
        tmp.index.set_names(["date", "ticker"], inplace=True)
        return tmp

    X_parts = [
        _stack_at(ret_21,  "ret_21"),
        _stack_at(ret_63,  "ret_63"),
        _stack_at(ret_126, "ret_126"),
        _stack_at(ret_252, "ret_252"),
        _stack_at(vol_63,  "vol_63"),
        _stack_at(vol_252, "vol_252"),
    ]

    if dvol_20 is not None:
        X_parts.append(_stack_at(dvol_20, "dvol_20"))

    px_sig = stock_close.loc[sig].stack(future_stack=True).rename("px").to_frame()
    px_sig.index.set_names(["date", "ticker"], inplace=True)
    X_parts.append(px_sig)

    X_price = pd.concat(X_parts, axis=1).sort_index()
    X_price = X_price[~X_price.index.duplicated(keep="last")]

    for c in X_price.columns:
        X_price[c] = X_price[c].astype("float32", copy=False)

    X_price.to_parquet(cache_path)
    print("Saved X_price:", cache_path)

print("X_price:", X_price.shape)
print("NA rate (top 12):")
print(X_price.isna().mean().sort_values(ascending=False).head(12))



Loaded cached X_price: (2326500, 8)
X_price: (2326500, 8)
NA rate (top 12):
ret_252    0.608837
vol_252    0.608837
ret_126    0.585915
ret_63     0.574187
vol_63     0.574187
ret_21     0.566300
dvol_20    0.563903
px         0.562341
dtype: float64


In [17]:
from pathlib import Path
import numpy as np
import pandas as pd

CACHE_DIR = Path(CACHE_DIR)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

DIM = "ARQ"
SF1_RAW_PATH = CACHE_DIR / f"sf1_raw_{DIM}_{pd.Timestamp(START).date()}_{pd.Timestamp(END).date()}.parquet"
SF1_ALIGNED_PATH = CACHE_DIR / f"sf1_factors_aligned_{DIM}_{pd.Timestamp(START).date()}_{pd.Timestamp(END).date()}.parquet"

INDICATORS_CSV_PATH = Path("SF1 Indicators.csv")

ind_meta = pd.read_csv(INDICATORS_CSV_PATH)
sf1_indicators = set(ind_meta.loc[ind_meta["table"].eq("SF1"), "indicator"].astype(str))

def _to_num(s):
    return pd.to_numeric(s, errors="coerce")

def _safe_div(a, b):
    a = _to_num(a)
    b = _to_num(b)
    return a / b.replace(0, np.nan)

def _safe_log_pos(x):
    x = _to_num(x)
    x = x.where(x > 0)
    return np.log(x)

def _neglog_pos(x):
    return -_safe_log_pos(x)

def _pct_change_4q(s):
    s = _to_num(s)
    return s.groupby(level=0).pct_change(4, fill_method=None)

def _diff_4q(s):
    s = _to_num(s)
    return s.groupby(level=0).diff(4)

def _shift_avail(d):
    return _shift_trading_days(all_days, d, FUNDAMENTALS_LAG_TRADING_DAYS)

def _col(name):
    return name if name in sf1_indicators else None

def _nonempty(s):
    s = pd.Series(s)
    return s.notna().any()

if SF1_RAW_PATH.exists():
    sf1_raw = pd.read_parquet(SF1_RAW_PATH)
else:
    sf1_start = (pd.Timestamp(START) - pd.Timedelta(days=365 * 2)).strftime("%Y-%m-%d")
    tickers_used = sorted(stock_close.columns.astype(str).tolist())
    sf1_raw = fetch_sf1_fundamentals_all_numeric(
        tickers=tickers_used,
        start=sf1_start,
        end=END,
        dimension=DIM,
        cache_name=f"sf1_raw_{DIM}_{pd.Timestamp(START).date()}_{pd.Timestamp(END).date()}",
    )
    sf1_raw.to_parquet(SF1_RAW_PATH)

sf1 = sf1_raw.copy()
sf1["ticker"] = sf1["ticker"].astype(str)
sf1["calendardate"] = pd.to_datetime(sf1.get("calendardate", pd.NaT), errors="coerce")
sf1["datekey"] = pd.to_datetime(sf1.get("datekey", pd.NaT), errors="coerce")
sf1 = sf1.dropna(subset=["ticker", "calendardate", "datekey"]).sort_values(["ticker", "calendardate"], kind="mergesort")

assets_c = _col("assets")
assetsavg_c = _col("assetsavg")
assetsc_c = _col("assetsc")
liabilitiesc_c = _col("liabilitiesc")
liabilities_c = _col("liabilities")
debt_c = _col("debt")
revenue_c = _col("revenue")
revenueusd_c = _col("revenueusd")
netinc_c = _col("netinc")
ncfo_c = _col("ncfo")
capex_c = _col("capex")
inventory_c = _col("inventory")
mcap_c = _col("marketcap")
epsdil_c = _col("epsdil")
eps_c = _col("eps")
epsusd_c = _col("epsusd")
divyield_c = _col("divyield")
pe_c = _col("pe")
pb_c = _col("pb")
ps_c = _col("ps")
evebitda_c = _col("evebitda")
grossmargin_c = _col("grossmargin")
ebitdamargin_c = _col("ebitdamargin")
netmargin_c = _col("netmargin")
currentratio_c = _col("currentratio")
assetturnover_c = _col("assetturnover")
roa_c = _col("roa")
roe_c = _col("roe")

sf1_keyed = sf1.set_index(["ticker", "calendardate"], drop=False)

assets_base = None
if assets_c and assets_c in sf1_keyed.columns:
    assets_base = sf1_keyed[assets_c]
elif assetsavg_c and assetsavg_c in sf1_keyed.columns:
    assets_base = sf1_keyed[assetsavg_c]

revenue_base = None
if revenue_c and revenue_c in sf1_keyed.columns:
    revenue_base = sf1_keyed[revenue_c]
elif revenueusd_c and revenueusd_c in sf1_keyed.columns:
    revenue_base = sf1_keyed[revenueusd_c]

eps_base = None
for nm in [epsdil_c, eps_c, epsusd_c]:
    if nm and nm in sf1_keyed.columns:
        eps_base = sf1_keyed[nm]
        break

val_pe = _neglog_pos(sf1_keyed[pe_c]) if (pe_c and pe_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
val_pb = _neglog_pos(sf1_keyed[pb_c]) if (pb_c and pb_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
val_ps = _neglog_pos(sf1_keyed[ps_c]) if (ps_c and ps_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
val_evebitda = _neglog_pos(sf1_keyed[evebitda_c]) if (evebitda_c and evebitda_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")

log_mcap = _safe_log_pos(sf1_keyed[mcap_c]) if (mcap_c and mcap_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")

grossmargin = _to_num(sf1_keyed[grossmargin_c]) if (grossmargin_c and grossmargin_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
ebitdamargin = _to_num(sf1_keyed[ebitdamargin_c]) if (ebitdamargin_c and ebitdamargin_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
netmargin = _to_num(sf1_keyed[netmargin_c]) if (netmargin_c and netmargin_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")

roa = _to_num(sf1_keyed[roa_c]) if (roa_c and roa_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
roe = _to_num(sf1_keyed[roe_c]) if (roe_c and roe_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
assetturnover = _to_num(sf1_keyed[assetturnover_c]) if (assetturnover_c and assetturnover_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
currentratio = _to_num(sf1_keyed[currentratio_c]) if (currentratio_c and currentratio_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")

rev_yoy = _pct_change_4q(revenue_base) if revenue_base is not None else pd.Series(index=sf1_keyed.index, dtype="float64")
eps_yoy = _pct_change_4q(eps_base) if eps_base is not None else pd.Series(index=sf1_keyed.index, dtype="float64")
asset_growth_yoy = _pct_change_4q(assets_base) if assets_base is not None else pd.Series(index=sf1_keyed.index, dtype="float64")

capex_assets = (
    _safe_div(sf1_keyed[capex_c], assets_base)
    if (capex_c and capex_c in sf1_keyed.columns_toggle if False else True)
    else pd.Series(index=sf1_keyed.index, dtype="float64")
)
# rebuild capex_assets correctly (avoid any accidental NameError)
if capex_c and capex_c in sf1_keyed.columns and assets_base is not None:
    capex_assets = _safe_div(sf1_keyed[capex_c], assets_base)
else:
    capex_assets = pd.Series(index=sf1_keyed.index, dtype="float64")

inv_growth_yoy = _pct_change_4q(sf1_keyed[inventory_c]) if (inventory_c and inventory_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
debt_growth_yoy = _pct_change_4q(sf1_keyed[debt_c]) if (debt_c and debt_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")
lev_debt_assets = _safe_div(sf1_keyed[debt_c], assets_base) if (debt_c and debt_c in sf1_keyed.columns and assets_base is not None) else pd.Series(index=sf1_keyed.index, dtype="float64")

accruals_ni_ncfo_assets = (
    _safe_div((_to_num(sf1_keyed[netinc_c]) - _to_num(sf1_keyed[ncfo_c])), assets_base)
    if (netinc_c and netinc_c in sf1_keyed.columns and ncfo_c and ncfo_c in sf1_keyed.columns and assets_base is not None)
    else pd.Series(index=sf1_keyed.index, dtype="float64")
)

if assetsc_c and assetsc_c in sf1_keyed.columns and liabilitiesc_c and liabilitiesc_c in sf1_keyed.columns and assets_base is not None:
    wc = _to_num(sf1_keyed[assetsc_c]) - _to_num(sf1_keyed[liabilitiesc_c])
    wc_chg_assets = _safe_div(_diff_4q(wc), assets_base)
else:
    wc_chg_assets = pd.Series(index=sf1_keyed.index, dtype="float64")

divyield = _to_num(sf1_keyed[divyield_c]) if (divyield_c and divyield_c in sf1_keyed.columns) else pd.Series(index=sf1_keyed.index, dtype="float64")

if (not _nonempty(roa)) and (netinc_c and netinc_c in sf1_keyed.columns) and (assets_base is not None):
    roa = _safe_div(sf1_keyed[netinc_c], assets_base)

if (not _nonempty(assetturnover)) and (revenue_base is not None) and (assets_base is not None):
    assetturnover = _safe_div(revenue_base, assets_base)

sf1_f = pd.DataFrame(
    {
        "ticker": sf1_keyed["ticker"].values,
        "datekey": sf1_keyed["datekey"].values,
        "calendardate": sf1_keyed["calendardate"].values,
        "val_pe": val_pe.values,
        "val_pb": val_pb.values,
        "val_ps": val_ps.values,
        "val_evebitda": val_evebitda.values,
        "log_mcap": log_mcap.values,
        "grossmargin": grossmargin.values,
        "ebitdamargin": ebitdamargin.values,
        "netmargin": netmargin.values,
        "roa": roa.values,
        "roe": roe.values,
        "currentratio": currentratio.values,
        "assetturnover": assetturnover.values,
        "rev_yoy": rev_yoy.values,
        "eps_yoy": eps_yoy.values,
        "asset_growth_yoy": asset_growth_yoy.values,
        "capex_assets": capex_assets.values,
        "inv_growth_yoy": inv_growth_yoy.values,
        "debt_growth_yoy": debt_growth_yoy.values,
        "lev_debt_assets": lev_debt_assets.values,
        "accruals_ni_ncfo_assets": accruals_ni_ncfo_assets.values,
        "wc_chg_assets": wc_chg_assets.values,
        "divyield": divyield.values,
    }
)

factor_cols_all = [
    "val_pe","val_pb","val_ps","val_evebitda",
    "log_mcap",
    "grossmargin","ebitdamargin","netmargin","roa","roe","currentratio","assetturnover",
    "rev_yoy","eps_yoy","asset_growth_yoy","capex_assets","inv_growth_yoy","debt_growth_yoy","lev_debt_assets",
    "accruals_ni_ncfo_assets","wc_chg_assets",
    "divyield",
]
factor_cols = [c for c in factor_cols_all if c in sf1_f.columns and _nonempty(sf1_f[c])]
print("Dropped all-NA factors:", [c for c in factor_cols_all if c not in factor_cols])

sf1_f["avail_date"] = pd.to_datetime(sf1_f["datekey"], errors="coerce").apply(_shift_avail)
sf1_f = sf1_f.dropna(subset=["ticker", "avail_date"])
sf1_f = sf1_f.sort_values(["ticker", "avail_date", "datekey"], kind="mergesort")
sf1_f = sf1_f.drop_duplicates(subset=["ticker", "avail_date"], keep="last")

if "X_price" in globals() and isinstance(X_price, pd.DataFrame):
    X = X_price.copy()
elif "X" in globals() and isinstance(X, pd.DataFrame):
    X = X.copy()
else:
    raise NameError("Need X_price (or X) defined before SF1 alignment.")

left = X.index.to_frame(index=False).copy()
left["date"] = pd.to_datetime(left["date"], errors="coerce")
left["ticker"] = left["ticker"].astype(str)
left = left.dropna(subset=["date", "ticker"])
left = left.drop_duplicates(subset=["date", "ticker"], keep="last")
left = left.sort_values(["date", "ticker"], kind="mergesort").reset_index(drop=True)

right = sf1_f[["ticker", "avail_date"] + factor_cols].copy()
right["ticker"] = right["ticker"].astype(str)
right["avail_date"] = pd.to_datetime(right["avail_date"], errors="coerce")
right = right.dropna(subset=["ticker", "avail_date"])
right = right.sort_values(["avail_date", "ticker"], kind="mergesort").reset_index(drop=True)

merged = pd.merge_asof(
    left,
    right,
    left_on="date",
    right_on="avail_date",
    by="ticker",
    direction="backward",
    allow_exact_matches=True,
)

lookahead = int((merged["avail_date"].notna() & (merged["avail_date"] > merged["date"])).sum())
print("Lookahead violations:", lookahead)

X_factors = merged.set_index(["date", "ticker"])[factor_cols].sort_index()
for c in X_factors.columns:
    X_factors[c] = X_factors[c].astype("float32", copy=False)

X_factors.to_parquet(SF1_ALIGNED_PATH)
print("SF1 aligned saved:", X_factors.shape, "to", SF1_ALIGNED_PATH)

X_full = X.join(X_factors, how="left").sort_index()
print("X_full:", X_full.shape)

na_rate = X_factors.isna().mean().sort_values(ascending=False)
print("NA rate (top 15):")
print(na_rate.head(15))


Dropped all-NA factors: ['roe']
Lookahead violations: 0
SF1 aligned saved: (2326500, 21) to cache_sharadar\sf1_factors_aligned_ARQ_2005-01-01_2025-12-31.parquet
X_full: (2326500, 29)
NA rate (top 15):
val_pe              0.574149
inv_growth_yoy      0.571228
val_evebitda        0.483093
wc_chg_assets       0.381220
currentratio        0.358379
debt_growth_yoy     0.339700
val_pb              0.314150
val_ps              0.281596
rev_yoy             0.272307
eps_yoy             0.257342
ebitdamargin        0.254324
netmargin           0.252348
grossmargin         0.252331
asset_growth_yoy    0.237264
divyield            0.232132
dtype: float64


In [19]:
# winsorize and make X panel
if "X_full" not in globals() or not isinstance(X_full, pd.DataFrame):
    raise NameError("X_full not found. Run the SF1 alignment cell first so X_full exists.")

factor_cols = [c for c in X_full.columns if c not in X_price.columns] if "X_price" in globals() else [c for c in X_full.columns if c not in X.columns]
factor_cols = [c for c in factor_cols if c in X_full.columns]

panel = X_full.copy()

def _winsorize_cs(df, lo=0.01, hi=0.99):
    qlo = df.groupby(level=0).quantile(lo)
    qhi = df.groupby(level=0).quantile(hi)

    out = df.copy()
    for c in df.columns:
        lo_s = qlo[c]
        hi_s = qhi[c]
        out[c] = out[c].clip(lower=lo_s, upper=hi_s)
    return out

def _zscore_cs(df):
    mu = df.groupby(level=0).transform("mean")
    sd = df.groupby(level=0).transform(lambda x: x.std(ddof=0))
    return (df - mu) / sd.replace(0, np.nan)

def _add_missing_indicators(df):
    miss = df.isna().astype("int8")
    miss.columns = [f"{c}__missing" for c in miss.columns]
    return miss

base_cols = [c for c in panel.columns if c not in factor_cols]
Xf = panel[factor_cols].astype("float32").copy()

coverage = Xf.notna().groupby(level=0).mean()
mean_coverage = coverage.mean().sort_values()
print("Mean factor coverage (bottom 10):")
print(mean_coverage.head(10))

min_daily_coverage = 0.05
keep_factors = mean_coverage[mean_coverage >= min_daily_coverage].index.tolist()
drop_factors = [c for c in factor_cols if c not in keep_factors]
print("Dropping low-coverage factors:", drop_factors)

Xf = Xf[keep_factors]

winsor_lo = 0.01
winsor_hi = 0.99
Xf_w = _winsorize_cs(Xf, lo=winsor_lo, hi=winsor_hi)

Xf_z = _zscore_cs(Xf_w)

miss_ind = _add_missing_indicators(Xf_z)

X_panel = pd.concat([panel[base_cols], Xf_z, miss_ind], axis=1).sort_index()

print("X_panel:", X_panel.shape)
print("Example columns:", X_panel.columns[:15].tolist())


Mean factor coverage (bottom 10):
val_pe             0.425851
inv_growth_yoy     0.428772
val_evebitda       0.516907
wc_chg_assets      0.618780
currentratio       0.641621
debt_growth_yoy    0.660300
val_pb             0.685850
val_ps             0.718404
rev_yoy            0.727693
eps_yoy            0.742658
dtype: float64
Dropping low-coverage factors: []


C:\Users\Admin\miniconda3\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\Admin\miniconda3\Lib\site-packages\numpy\_core\_methods.py:51: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


X_panel: (2326500, 50)
Example columns: ['ret_21', 'ret_63', 'ret_126', 'ret_252', 'vol_63', 'vol_252', 'dvol_20', 'px', 'val_pe', 'inv_growth_yoy', 'val_evebitda', 'wc_chg_assets', 'currentratio', 'debt_growth_yoy', 'val_pb']


In [20]:
# establish 3 month total return for backtest

if "px_stocks" not in globals() or not isinstance(px_stocks, pd.DataFrame):
    raise NameError("px_stocks not found. Load SEP first so px_stocks exists with columns ['ticker','date', ...].")

if "X_panel" not in globals() or not isinstance(X_panel, pd.DataFrame):
    raise NameError("X_panel not found. Run your winsorize/zscore chunk first so X_panel exists with MultiIndex (date, ticker).")

px = px_stocks.copy()
px["date"] = pd.to_datetime(px["date"], errors="coerce")
px["ticker"] = px["ticker"].astype(str)

close_col = "close" if "close" in px.columns else ("Close" if "Close" in px.columns else None)
if close_col is None:
    raise KeyError(f"Couldn't find a close column in px_stocks. Columns: {px.columns.tolist()}")

px = px.dropna(subset=["ticker", "date", close_col]).sort_values(["ticker", "date"], kind="mergesort")

px_wide = px.pivot(index="date", columns="ticker", values=close_col).sort_index()
px_wide = px_wide.ffill()

h = 63  # ~3 months trading days
fwd_ret = px_wide.shift(-h) / px_wide - 1.0

y = fwd_ret.stack(dropna=False)
y.index.names = ["date", "ticker"]
y.name = "ret_3m_fwd"

panel_index = X_panel.index
y = y.reindex(panel_index)

mask = y.notna()
print("y coverage:", float(mask.mean()), "non-NA:", int(mask.sum()), "total:", int(mask.size))
print("y sample:", y.dropna().iloc[:5].to_dict())

y_ret3m = y


C:\Users\Admin\AppData\Local\Temp\ipykernel_23624\2434841225.py:25: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  y = fwd_ret.stack(dropna=False)


y coverage: 0.7835529765742532 non-NA: 1822936 total: 2326500
y sample: {(Timestamp('2005-01-31 00:00:00'), 'A'): -0.05156037991858886, (Timestamp('2005-01-31 00:00:00'), 'AABC'): -0.0034602076124566894, (Timestamp('2005-01-31 00:00:00'), 'AACC'): 0.02006707975669375, (Timestamp('2005-01-31 00:00:00'), 'AACE'): -0.10984848484848475, (Timestamp('2005-01-31 00:00:00'), 'AAI'): -0.03161592505854793}


In [27]:
if "X_panel" not in globals() or not isinstance(X_panel, pd.DataFrame):
    raise NameError("X_panel not found.")
if "y_ret3m" not in globals():
    raise NameError("y_ret3m not found.")

dates = pd.to_datetime(X_panel.index.get_level_values("date")).unique()
dates = pd.DatetimeIndex(sorted(dates))

q_end_dates = pd.DatetimeIndex(sorted(dates[dates.is_quarter_end]))

print("Quarter-end decision dates:", len(q_end_dates))
print("First / last q-end:", q_end_dates.min(), q_end_dates.max())

train_q = 32
val_q = 8
test_q = 12
step_q = 4

folds = []
i = 0
while True:
    train_start = i
    train_end = train_start + train_q
    val_end = train_end + val_q
    test_end = val_end + test_q

    if test_end > len(q_end_dates):
        break

    folds.append(
        {
            "train": q_end_dates[train_start:train_end],
            "val": q_end_dates[train_end:val_end],
            "test": q_end_dates[val_end:test_end],
        }
    )
    i += step_q

for k, f in enumerate(folds, 1):
    print(f"\nFold {k}")
    print("  Train:", f["train"][0].date(), "->", f["train"][-1].date(), f"({len(f['train'])} q)")
    print("  Val:  ", f["val"][0].date(), "->", f["val"][-1].date(), f"({len(f['val'])} q)")
    print("  Test: ", f["test"][0].date(), "->", f["test"][-1].date(), f"({len(f['test'])} q)")

mask_q = X_panel.index.get_level_values("date").isin(q_end_dates)
Xq = X_panel.loc[mask_q].copy()
yq = y_ret3m.reindex(Xq.index)

print("\nQuarterly X shape:", Xq.shape)
print("Quarterly y coverage:", float(yq.notna().mean()))

fold_datasets = []
for k, f in enumerate(folds, 1):
    d = Xq.index.get_level_values("date")
    train_mask = d.isin(f["train"])
    val_mask = d.isin(f["val"])
    test_mask = d.isin(f["test"])

    fold_datasets.append(
        {
            "fold": k,
            "train_dates": f["train"],
            "val_dates": f["val"],
            "test_dates": f["test"],
            "X_train": Xq.loc[train_mask],
            "y_train": yq.loc[train_mask],
            "X_val": Xq.loc[val_mask],
            "y_val": yq.loc[val_mask],
            "X_test": Xq.loc[test_mask],
            "y_test": yq.loc[test_mask],
        }
    )

    print(f"\nFold {k} rows")
    print("  Train rows:", int(train_mask.sum()), "y non-NA:", int(yq.loc[train_mask].notna().sum()))
    print("  Val rows:  ", int(val_mask.sum()), "y non-NA:", int(yq.loc[val_mask].notna().sum()))
    print("  Test rows: ", int(test_mask.sum()), "y non-NA:", int(yq.loc[test_mask].notna().sum()))


Quarter-end decision dates: 58
First / last q-end: 2005-03-31 00:00:00 2025-09-30 00:00:00

Fold 1
  Train: 2005-03-31 -> 2015-12-31 (32 q)
  Val:   2016-03-31 -> 2019-12-31 (8 q)
  Test:  2020-03-31 -> 2023-03-31 (12 q)

Fold 2
  Train: 2006-06-30 -> 2017-03-31 (32 q)
  Val:   2017-06-30 -> 2020-12-31 (8 q)
  Test:  2021-03-31 -> 2025-03-31 (12 q)

Quarterly X shape: (539748, 50)
Quarterly y coverage: 0.776664295189607

Fold 1 rows
  Train rows: 297792 y non-NA: 207451
  Val rows:   74448 y non-NA: 62611
  Test rows:  111672 y non-NA: 103553

Fold 2 rows
  Train rows: 297792 y non-NA: 216172
  Val rows:   74448 y non-NA: 65491
  Test rows:  111672 y non-NA: 106646


In [33]:
#backtest

import random
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm.auto import tqdm

if "fold_datasets" not in globals() or len(fold_datasets) == 0:
    raise NameError("fold_datasets not found. Run the combined folds+datasets cell first.")

if "Xq" not in globals() or "yq" not in globals():
    raise NameError("Xq/yq not found. Run the combined folds+datasets cell first.")

SEED = int(globals().get("SEED", 42))
N_LONG = int(globals().get("N_LONG", 50))
N_SHORT = int(globals().get("N_SHORT", 50))
COST_BPS_ONE_WAY = float(globals().get("COST_BPS_ONE_WAY", 0.0))

np.random.seed(SEED)
random.seed(SEED)

def _cs_zscore_series(y_s):
    df = y_s.rename("y").reset_index()
    m = df.groupby("date")["y"].transform("mean")
    s = df.groupby("date")["y"].transform("std").replace(0.0, np.nan)
    out = (df["y"] - m) / s
    out.index = pd.MultiIndex.from_frame(df[["date", "ticker"]])
    out.index.names = ["date", "ticker"]
    return out

def _spearman_ic(pred_s, y_s):
    df = pd.DataFrame({"p": pred_s, "y": y_s}).dropna()
    if df.empty:
        return np.nan
    return df["p"].rank().corr(df["y"].rank())

def _best_ntree(booster):
    try:
        a = booster.attributes()
        if "best_iteration" in a:
            return int(a["best_iteration"]) + 1
    except Exception:
        pass
    return None

def _stable_top_bottom(pred_s, n_long, n_short):
    s = pred_s.dropna()
    if s.empty:
        return [], []
    idx = s.index.astype(str).to_numpy()
    val = s.to_numpy()
    order_long = np.lexsort((idx, -val))
    order_short = np.lexsort((idx, val))
    longs = s.index.values[order_long][:n_long].tolist()
    shorts = s.index.values[order_short][:n_short].tolist()
    return longs, shorts

base_params = {
    "max_depth": 4,
    "eta": 0.03,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "lambda": 1.0,
    "alpha": 0.0,
    "seed": SEED,
    "nthread": 1,
    "tree_method": "hist",
    "predictor": "cpu_predictor",
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
}

feat_cols = list(Xq.columns)
tickers_all = pd.Index(sorted(Xq.index.get_level_values("ticker").astype(str).unique().tolist()))
prev_w = pd.Series(0.0, index=tickers_all)

rows_fit = []
rows_perf = []
rows_w = []
rows_hold = []

tasks = []
for fd in fold_datasets:
    test_dates = pd.Index(sorted(fd["X_test"].index.get_level_values("date").unique()))
    for d in test_dates:
        tasks.append((int(fd["fold"]), pd.Timestamp(d)))

pbar = tqdm(tasks, desc="Quarterly OOS quarters", unit="q", mininterval=0.5)

fold_map = {int(fd["fold"]): fd for fd in fold_datasets}
trained_models = {}

for fold_k, sig in pbar:
    fd = fold_map[fold_k]

    if fold_k not in trained_models:
        X_train = fd["X_train"].sort_index()
        y_train = fd["y_train"].reindex(X_train.index)

        X_val = fd["X_val"].sort_index()
        y_val = fd["y_val"].reindex(X_val.index)

        y_train_cs = _cs_zscore_series(y_train)
        y_val_cs = _cs_zscore_series(y_val)

        mtr = y_train_cs.notna()
        mva = y_val_cs.notna()

        Xtr = X_train.loc[mtr, feat_cols].to_numpy(dtype=np.float32, copy=False)
        ytr = y_train_cs.loc[mtr].to_numpy(dtype=np.float32, copy=False)
        Xva = X_val.loc[mva, feat_cols].to_numpy(dtype=np.float32, copy=False)
        yva = y_val_cs.loc[mva].to_numpy(dtype=np.float32, copy=False)

        if len(ytr) == 0 or len(yva) == 0:
            trained_models[fold_k] = None
            continue

        dtr = xgb.DMatrix(Xtr, label=ytr, feature_names=feat_cols)
        dva = xgb.DMatrix(Xva, label=yva, feature_names=feat_cols)

        evals_result = {}
        booster = xgb.train(
            params=base_params,
            dtrain=dtr,
            num_boost_round=5000,
            evals=[(dva, "val")],
            early_stopping_rounds=100,
            verbose_eval=False,
            evals_result=evals_result,
        )

        nt = _best_ntree(booster)
        try:
            n_trained = int(booster.num_boosted_rounds())
        except Exception:
            n_trained = np.nan

        val_rmse_path = evals_result.get("val", {}).get("rmse", [])
        val_rmse_best = float(np.min(val_rmse_path)) if len(val_rmse_path) else np.nan
        val_rmse_last = float(val_rmse_path[-1]) if len(val_rmse_path) else np.nan

        val_pred = booster.predict(dva)
        rmse_model = float(np.sqrt(np.mean((val_pred - yva) ** 2)))
        rmse_zero = float(np.sqrt(np.mean((0.0 - yva) ** 2)))
        rmse_improve = rmse_zero - rmse_model
        ic = _spearman_ic(pd.Series(val_pred), pd.Series(yva))

        rows_fit.append(
            {
                "fold": fold_k,
                "trained_rounds": n_trained,
                "best_trees": int(nt) if nt is not None else np.nan,
                "val_rmse_best": val_rmse_best,
                "val_rmse_last": val_rmse_last,
                "rmse_model": rmse_model,
                "rmse_zero": rmse_zero,
                "rmse_improve": rmse_improve,
                "val_ic": float(ic) if ic == ic else np.nan,
                "train_rows": int(len(ytr)),
                "val_rows": int(len(yva)),
            }
        )

        trained_models[fold_k] = {
            "booster": booster,
            "nt": nt,
            "n_trained": n_trained,
            "val_ic": float(ic) if ic == ic else np.nan,
            "rmse_improve": rmse_improve,
        }

    model_pack = trained_models.get(fold_k)
    if model_pack is None:
        continue

    booster = model_pack["booster"]
    nt = model_pack["nt"]
    n_trained = model_pack["n_trained"]
    ic = model_pack["val_ic"]
    rmse_improve = model_pack["rmse_improve"]

    X_test = fd["X_test"].sort_index()
    y_test = fd["y_test"].reindex(X_test.index)

    try:
        Xsig = X_test.loc[(sig, slice(None)), feat_cols]
        ysig = y_test.loc[(sig, slice(None))]
    except KeyError:
        continue

    if Xsig.empty:
        continue

    Xsig = Xsig.sort_index()
    Xsig_np = Xsig.to_numpy(dtype=np.float32, copy=False)
    pred = pd.Series(
        booster.predict(xgb.DMatrix(Xsig_np, feature_names=feat_cols)),
        index=Xsig.index.get_level_values("ticker").astype(str),
    )

    ysig = ysig.copy()
    ysig.index = ysig.index.get_level_values("ticker").astype(str)

    universe0 = pred.index.intersection(ysig.index)

    MIN_PRICE = float(globals().get("MIN_PRICE", 5.0))
    MIN_DVOL_20 = float(globals().get("MIN_DVOL_20", 1e6))
    
    core = Xsig.copy()
    core.index = core.index.get_level_values("ticker").astype(str)
    
    px_ok = core["px"] >= MIN_PRICE if "px" in core.columns else pd.Series(True, index=core.index)
    liq_ok = core["dvol_20"] >= MIN_DVOL_20 if "dvol_20" in core.columns else pd.Series(True, index=core.index)
    
    universe = [t for t in universe0 if bool(px_ok.get(t, False)) and bool(liq_ok.get(t, False))]
    
    pred_u = pred.loc[universe].dropna()
    y_u = ysig.loc[universe]
    
    if len(pred_u) < (N_LONG + N_SHORT):
        continue
    
    longs, shorts = _stable_top_bottom(pred_u, N_LONG, N_SHORT)
    if len(longs) == 0 or len(shorts) == 0:
        continue

    w = pd.Series(0.0, index=tickers_all)
    w.loc[longs] = 1.0 / len(longs)
    w.loc[shorts] = -1.0 / len(shorts)

    held = longs + shorts
    y_held = y_u.reindex(held).astype("float64").dropna()
    if y_held.empty:
        continue

    w_held = w.reindex(y_held.index).astype("float64")
    gross = float((w_held * y_held).sum())

    turnover = 0.5 * float((w - prev_w).abs().sum())
    cost = turnover * (COST_BPS_ONE_WAY / 1e4)
    net = gross - cost

    rows_perf.append(
        {
            "fold": fold_k,
            "signal_date": pd.Timestamp(sig),
            "gross_ret": gross,
            "net_ret": net,
            "turnover": turnover,
            "n_universe": int(len(pred_u)),
            "trained_rounds": n_trained,
            "best_trees": int(nt) if nt is not None else np.nan,
            "val_ic": float(ic) if ic == ic else np.nan,
            "rmse_improve": rmse_improve,
        }
    )

    w_sig = w[w != 0.0]
    for tk, ww in w_sig.items():
        rows_w.append({"fold": fold_k, "signal_date": pd.Timestamp(sig), "ticker": str(tk), "weight": float(ww)})

    for tk in y_held.index:
        side = "LONG" if w.loc[tk] > 0 else "SHORT"
        rows_hold.append(
            {
                "fold": fold_k,
                "signal_date": pd.Timestamp(sig),
                "ticker": str(tk),
                "side": side,
                "weight": float(w.loc[tk]),
                "asset_return": float(y_held.loc[tk]),
                "pred": float(pred_u.loc[tk]) if tk in pred_u.index else np.nan,
            }
        )

    prev_w = w
    pbar.set_postfix_str(f"fold={fold_k} date={sig.date()}")

pbar.close()

perf = pd.DataFrame(rows_perf).sort_values(["signal_date", "fold"]).set_index("signal_date")
fit_diag = pd.DataFrame(rows_fit).sort_values(["fold"]).reset_index(drop=True)
weights = pd.DataFrame(rows_w).sort_values(["signal_date", "fold", "ticker"]).reset_index(drop=True)
holdings = pd.DataFrame(rows_hold).sort_values(["signal_date", "fold", "side", "ticker"]).reset_index(drop=True)

def _perf_stats(r):
    r = pd.Series(r).dropna()
    if r.empty:
        return pd.Series(dtype="float64")
    ann_factor = 4.0
    n = float(len(r))
    equity = (1.0 + r).cumprod()
    ann_ret = equity.iloc[-1] ** (ann_factor / n) - 1.0
    ann_vol = r.std(ddof=0) * np.sqrt(ann_factor)
    sharpe = (r.mean() / r.std(ddof=0) * np.sqrt(ann_factor)) if r.std(ddof=0) != 0 else np.nan
    running_max = equity.cummax()
    dd = equity / running_max - 1.0
    mdd = dd.min()
    hit = (r > 0).mean()
    return pd.Series(
        {
            "n_periods": n,
            "ann_return": float(ann_ret),
            "ann_vol": float(ann_vol),
            "sharpe": float(sharpe) if sharpe == sharpe else np.nan,
            "max_drawdown": float(mdd),
            "hit_rate": float(hit),
            "avg_period_return": float(r.mean()),
            "median_period_return": float(r.median()),
            "cum_return": float(equity.iloc[-1] - 1.0),
        }
    )

stats_net = _perf_stats(perf["net_ret"]) if "net_ret" in perf.columns else pd.Series(dtype="float64")
stats_gross = _perf_stats(perf["gross_ret"]) if "gross_ret" in perf.columns else pd.Series(dtype="float64")

print("perf rows:", len(perf), "fit folds:", len(fit_diag), "weights rows:", len(weights), "holdings rows:", len(holdings))
print("\nBacktest stats (NET):")
print(stats_net)
print("\nBacktest stats (GROSS):")
print(stats_gross)

perf.tail(), fit_diag, holdings.head(), weights.head()


Quarterly OOS quarters:   0%|                                                                    | 0/24 [00:00<?, ?q/s]C:\Users\Admin\miniconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [16:37:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "predictor" } are not used.

  self.starting_round = model.num_boosted_rounds()
Quarterly OOS quarters:  29%|██████████▌                         | 7/24 [00:06<00:11,  1.45q/s, fold=1 date=2023-03-31]C:\Users\Admin\miniconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [16:37:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "predictor" } are not used.

  self.starting_round = model.num_boosted_rounds()
Quarterly OOS quarters: 100%|███████████████████████████████████| 24/24 [00:20<00:00,  1.17q/s, fold=2 date=2025-03-31]

perf rows: 24 fit folds: 2 weights rows: 2400 holdings rows: 2400

Backtest stats (NET):
n_periods               24.000000
ann_return               0.411789
ann_vol                  0.345616
sharpe                   1.206711
max_drawdown            -0.206253
hit_rate                 0.666667
avg_period_return        0.104265
median_period_return     0.114979
cum_return               6.918058
dtype: float64

Backtest stats (GROSS):
n_periods               24.000000
ann_return               0.414794
ann_vol                  0.345611
sharpe                   1.213350
max_drawdown            -0.205653
hit_rate                 0.666667
avg_period_return        0.104837
median_period_return     0.115564
cum_return               7.019721
dtype: float64


(             fold  gross_ret   net_ret  turnover  n_universe  trained_rounds  \
 signal_date                                                                    
 2023-03-31      2  -0.052685 -0.053285      1.20        1785             423   
 2023-06-30      2   0.103997  0.103367      1.26        1803             423   
 2024-09-30      2   0.011251  0.010501      1.50        1757             423   
 2024-12-31      2   0.102179  0.101629      1.10        1802             423   
 2025-03-31      2  -0.205653 -0.206253      1.20        1759             423   
 
              best_trees    val_ic  rmse_improve  
 signal_date                                      
 2023-03-31          323 -0.008934      0.020526  
 2023-06-30          323 -0.008934      0.020526  
 2024-09-30          323 -0.008934      0.020526  
 2024-12-31          323 -0.008934      0.020526  
 2025-03-31          323 -0.008934      0.020526  ,
    fold  trained_rounds  best_trees  val_rmse_best  val_rmse_last  rmse_

In [34]:
# IC time series

if "holdings" not in globals():
    raise NameError("holdings not found.")

ic_rows = []
for (d, f), grp in holdings.groupby(["signal_date","fold"]):
    if grp.shape[0] < 50:
        continue
    ic = grp["pred"].rank().corr(grp["asset_return"].rank())
    ic_rows.append({"signal_date": d, "fold": f, "ic": ic})

ic_df = pd.DataFrame(ic_rows).sort_values("signal_date")
print("Mean IC:", ic_df["ic"].mean())
print("IC std:", ic_df["ic"].std())
print("IC t-stat:", ic_df["ic"].mean() / (ic_df["ic"].std() / np.sqrt(len(ic_df))))

ic_df.tail()


Mean IC: 0.23768586119893934
IC std: 0.27136433177548264
IC t-stat: 4.290977190717072


,signal_date,fold,ic
19,2023-03-31,2,-0.082616
20,2023-06-30,2,0.175976
21,2024-09-30,2,-0.018758
22,2024-12-31,2,0.246853
23,2025-03-31,2,-0.256166


In [38]:
#decile test

decile_rows = []

for (d, f), grp in holdings.groupby(["signal_date","fold"]):
    grp = grp.dropna(subset=["pred","asset_return"])
    if len(grp) < 100:
        continue

    grp["decile"] = pd.qcut(grp["pred"], 10, labels=False, duplicates="drop")
    dec = grp.groupby("decile")["asset_return"].mean()

    for k, v in dec.items():
        decile_rows.append({"signal_date": d, "fold": f, "decile": int(k), "ret": v})

dec_df = pd.DataFrame(decile_rows)
dec_summary = dec_df.groupby("decile")["ret"].mean()
print(dec_summary)


               signal_date       fold  ls_spread
count                   24  24.000000  24.000000
mean   2022-04-23 05:00:00   1.500000   0.108251
min    2020-03-31 00:00:00   1.000000  -0.157342
25%    2021-06-07 06:00:00   1.000000  -0.049100
50%    2022-02-14 00:00:00   1.500000   0.067542
75%    2022-11-14 12:00:00   2.000000   0.208911
max    2025-03-31 00:00:00   2.000000   0.529449
std                    NaN   0.510754   0.191699


In [39]:
# long short spread
ls_rows = []

for (d, f), grp in holdings.groupby(["signal_date","fold"]):
    grp = grp.dropna(subset=["pred","asset_return"])
    if len(grp) < 100:
        continue

    grp["decile"] = pd.qcut(grp["pred"], 10, labels=False, duplicates="drop")
    top = grp[grp["decile"] == grp["decile"].max()]["asset_return"].mean()
    bot = grp[grp["decile"] == grp["decile"].min()]["asset_return"].mean()

    ls_rows.append({
        "signal_date": d,
        "fold": f,
        "ls_spread": top - bot
    })

ls_df = pd.DataFrame(ls_rows)
print(ls_df.describe())


               signal_date       fold  ls_spread
count                   24  24.000000  24.000000
mean   2022-04-23 05:00:00   1.500000   0.108251
min    2020-03-31 00:00:00   1.000000  -0.157342
25%    2021-06-07 06:00:00   1.000000  -0.049100
50%    2022-02-14 00:00:00   1.500000   0.067542
75%    2022-11-14 12:00:00   2.000000   0.208911
max    2025-03-31 00:00:00   2.000000   0.529449
std                    NaN   0.510754   0.191699


In [36]:
perf_reset = perf.reset_index()
perf_reset["year"] = perf_reset["signal_date"].dt.year

print("Pre-2015:")
print(perf_reset[perf_reset["year"] < 2015]["net_ret"].mean())

print("2015–2019:")
print(perf_reset[(perf_reset["year"] >= 2015) & (perf_reset["year"] < 2020)]["net_ret"].mean())

print("2020+ :")
print(perf_reset[perf_reset["year"] >= 2020]["net_ret"].mean())


Pre-2015:
nan
2015–2019:
nan
2020+ :
0.1042647003186491


In [37]:
importances = []

for fd in fold_datasets:
    k = fd["fold"]
    model = trained_models.get(k, {}).get("booster")
    if model is None:
        continue
    imp = model.get_score(importance_type="gain")
    for f, v in imp.items():
        importances.append({"fold": k, "feature": f, "gain": v})

imp_df = pd.DataFrame(importances)
imp_avg = imp_df.groupby("feature")["gain"].mean().sort_values(ascending=False)
imp_avg.head(15)


feature
assetturnover               436.531479
val_ps                      362.882805
ebitdamargin                318.630547
val_evebitda                305.299355
dvol_20                     288.611992
capex_assets                272.133812
asset_growth_yoy            268.998268
lev_debt_assets__missing    254.822060
ret_63                      248.054756
val_evebitda__missing       246.002060
eps_yoy                     244.078262
val_pb                      227.288643
rev_yoy                     220.118164
wc_chg_assets               217.192574
ret_21                      215.994720
Name: gain, dtype: float64

In [ ]:
#stats and export
import numpy as np
import pandas as pd
from pathlib import Path

if "perf" not in globals() or perf.empty:
    raise NameError("perf not found or empty.")
if "weights" not in globals() or weights.empty:
    raise NameError("weights not found or empty.")
if "holdings" not in globals() or holdings.empty:
    raise NameError("holdings not found or empty.")

perf_reset = perf.reset_index().copy()
perf_reset["signal_date"] = pd.to_datetime(perf_reset["signal_date"])
perf_reset["fold"] = perf_reset["fold"].astype(int)

perf_live = perf_reset.sort_values(["signal_date","fold"]).drop_duplicates(subset=["signal_date"], keep="last").set_index("signal_date").sort_index()

weights_live = weights.copy()
weights_live["signal_date"] = pd.to_datetime(weights_live["signal_date"])
weights_live["fold"] = weights_live["fold"].astype(int)
weights_live = weights_live.sort_values(["signal_date","fold","ticker"]).drop_duplicates(subset=["signal_date","ticker"], keep="last").reset_index(drop=True)

holdings_live = holdings.copy()
holdings_live["signal_date"] = pd.to_datetime(holdings_live["signal_date"])
holdings_live["fold"] = holdings_live["fold"].astype(int)
holdings_live = holdings_live.sort_values(["signal_date","fold","ticker"]).drop_duplicates(subset=["signal_date","ticker"], keep="last").reset_index(drop=True)

def _perf_stats(r):
    r = pd.Series(r).dropna()
    if r.empty:
        return pd.Series(dtype="float64")
    ann_factor = 4.0
    n = float(len(r))
    equity = (1.0 + r).cumprod()
    ann_ret = equity.iloc[-1] ** (ann_factor / n) - 1.0
    ann_vol = r.std(ddof=0) * np.sqrt(ann_factor)
    sharpe = (r.mean() / r.std(ddof=0) * np.sqrt(ann_factor)) if r.std(ddof=0) != 0 else np.nan
    running_max = equity.cummax()
    dd = equity / running_max - 1.0
    mdd = dd.min()
    hit = (r > 0).mean()
    return pd.Series(
        {
            "n_quarters": n,
            "ann_return": float(ann_ret),
            "ann_vol": float(ann_vol),
            "sharpe": float(sharpe) if sharpe == sharpe else np.nan,
            "max_drawdown": float(mdd),
            "hit_rate": float(hit),
            "avg_q_return": float(r.mean()),
            "median_q_return": float(r.median()),
            "cum_return": float(equity.iloc[-1] - 1.0),
        }
    )

print("perf rows (all folds):", len(perf_reset))
print("perf rows (live dedup):", len(perf_live))

print("\nLIVE Backtest stats (NET):")
print(_perf_stats(perf_live["net_ret"]))

print("\nLIVE Backtest stats (GROSS):")
print(_perf_stats(perf_live["gross_ret"]))

export_date = perf_live.index.max()
print("\nExporting holdings for:", export_date.date())

wq = weights_live[weights_live["signal_date"] == export_date].copy()
hq = holdings_live[holdings_live["signal_date"] == export_date].copy()

longs = hq[hq["side"] == "LONG"][["ticker","weight","asset_return"]].sort_values("weight", ascending=False).head(50).reset_index(drop=True)
shorts = hq[hq["side"] == "SHORT"][["ticker","weight","asset_return"]].sort_values("weight", ascending=True).head(50).reset_index(drop=True)

out_path = Path(CACHE_DIR) / f"quarterly_holdings_{export_date.date()}_LIVE.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    longs.to_excel(writer, sheet_name="LONG", index=False)
    shorts.to_excel(writer, sheet_name="SHORT", index=False)

print("Saved:", out_path)

perf_live.tail()
